In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
#import random
#import sklearn
#import collections
#from sklearn.model_selection import train_test_split
#import json
#import pylab 
#from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import matplotlib
import matplotlib.patches as mpatches
#import shap
#import pandas as pd
import tensorflow as tf
#import tarfile
from tensorflow.keras.models import load_model
#from qkeras import QActivation, QDense, QConv2D, QBatchNormalization
import ensembler_functions as ef
import tf2onnx
import onnx
import os
from sklearn.model_selection import train_test_split
import load_and_match as lam

2024-11-12 15:24:16.244161: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-12 15:24:16.335722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Welcome to JupyROOT 6.30/04


In [3]:
datasets = {}

In [4]:
# Collect new EB data that will have incorrect ordering.

# Define the base path where the h5 files are stored
base_path = '/eos/home-m/mmcohen/ntuples/EB_h5_10-06-2024/'

# Initialize empty lists to collect arrays from all files
HLT_jets_list = []
ofl_jets_list = []
L1_jFexSR_jets_list = []
L1_jFexLR_jets_list = []
HLT_electrons_list = []
LRT_electrons_list = []
ofl_electrons_list = []
L1_egammas_list = []
HLT_muons_list = []
LRT_muons_list = []
ofl_muons_list = []
L1_muons_list = []
L1_eFex_taus_list = []
L1_jFex_taus_list = []
HLT_photons_list = []
ofl_photons_list = []
HLT_MET_list = []
L1_MET_list = []
pass_L1_unprescaled_list = []
pass_HLT_unprescaled_list = []
EB_weights_list = []
event_number_list = []
run_number_list = []

# Iterate over all files in the directory
for file_name in os.listdir(base_path):
    if (file_name.startswith('EB') == False): continue
    file_path = os.path.join(base_path, file_name)
    
    # Open each h5 file and append data to lists
    with h5py.File(file_path, 'r') as hf:
        HLT_jets = hf['HLT_jets'][:]
        ofl_jets = hf['ofl_jets'][:]
        L1_jFexSR_jets = hf['L1_jFexSR_jets'][:]
        L1_jFexLR_jets = hf['L1_jFexLR_jets'][:]
        HLT_electrons = hf['HLT_electrons'][:]
        LRT_electrons = hf['LRT_electrons'][:]
        ofl_electrons = hf['ofl_electrons'][:]
        L1_egammas = hf['L1_egammas'][:]
        HLT_muons = hf['HLT_muons'][:]
        LRT_muons = hf['LRT_muons'][:]
        ofl_muons = hf['ofl_muons'][:]
        L1_muons = hf['L1_muons'][:]
        L1_eFex_taus = hf['L1_eFex_taus'][:]
        L1_jFex_taus = hf['L1_jFex_taus'][:]
        HLT_photons = hf['HLT_photons'][:]
        ofl_photons = hf['ofl_photons'][:]
        HLT_MET = hf['HLT_MET'][:].reshape(-1, 1, 3)  # Broadcasting MET
        L1_MET = hf['L1_MET'][:].reshape(-1, 1, 3)
        pass_L1_unprescaled = hf["pass_L1_unprescaled"][:]
        pass_HLT_unprescaled = hf["pass_HLT_unprescaled"][:]
        EB_weights = hf["EB_weights"][:]
        event_number = hf["event_number"][:]
        run_number = hf["run_number"][:]
        mu = hf["mu"][:]

    HLT_objects = np.concatenate([HLT_jets[:, :6, [0, 2, 3]], HLT_electrons[:, :3, :], HLT_muons[:, :3, :], HLT_photons[:, :3, :], HLT_MET], axis=1)
    L1_objects = np.concatenate([L1_jFexSR_jets[:, :6, :], L1_egammas[:, :3, :], L1_muons[:, :3, :], L1_eFex_taus[:, :3, :], L1_MET], axis=1)
    
    datasets[file_name.split('_10')[0]] = {
        'HLT_data': HLT_objects,
        'L1_data': L1_objects,
        'passL1': pass_L1_unprescaled==1,
        'passHLT': pass_HLT_unprescaled==1,
        'weights': EB_weights,
        'event_numbers': event_number,
        'run_numbers': run_number,
        'pileups': mu
    }

def combine_data(datasets, tags_to_combine, new_tag):

    # initialize empty lists for new tag
    datasets[new_tag] = {key: [] for key in datasets[tags_to_combine[0]].keys()}

    # Loop through old tags and append np arrays to lists
    for tag in tags_to_combine:
        for key, value in datasets[tag].items():
            datasets[new_tag][key].append(value)

    # Concatenate lists into single np array
    for key, value in datasets[new_tag].items():
        datasets[new_tag][key] = np.concatenate(value, axis=0)

    # Delete old tags
    for tag in tags_to_combine:
        del datasets[tag]

    # Make sure everything is an np array
    for tag, data_dict in datasets.items():
        for key, value in data_dict.items():
            data_dict[key] = np.array(value)

    return datasets

datasets = combine_data(datasets, tags_to_combine=['EB_475341_0', 'EB_475341_1'], new_tag='HLT_noalg_eb_L1All')
#datasets = combine_data(datasets, tags_to_combine=['EB_473255_0', 'EB_475321_0', 'EB_482596_0'], new_tag='EB')
#datasets = combine_data(datasets, tags_to_combine=['EB_473255_0', 'EB_475321_0'], new_tag='EB')

In [5]:
for tag, data_dict in datasets.items():
        print(f'{tag}:')
        for key, value in data_dict.items():
            print(f'    {key}: {value.shape}')

EB_473255_0:
    HLT_data: (1146112, 16, 3)
    L1_data: (1146112, 16, 3)
    passL1: (1146112,)
    passHLT: (1146112,)
    weights: (1146112,)
    event_numbers: (1146112,)
    run_numbers: (1146112,)
    pileups: (1146112,)
EB_475321_0:
    HLT_data: (1119657, 16, 3)
    L1_data: (1119657, 16, 3)
    passL1: (1119657,)
    passHLT: (1119657,)
    weights: (1119657,)
    event_numbers: (1119657,)
    run_numbers: (1119657,)
    pileups: (1119657,)
EB_482596_0:
    HLT_data: (1048336, 16, 3)
    L1_data: (1048336, 16, 3)
    passL1: (1048336,)
    passHLT: (1048336,)
    weights: (1048336,)
    event_numbers: (1048336,)
    run_numbers: (1048336,)
    pileups: (1048336,)
HLT_noalg_eb_L1All:
    HLT_data: (1107321, 16, 3)
    L1_data: (1107321, 16, 3)
    passL1: (1107321,)
    passHLT: (1107321,)
    weights: (1107321,)
    event_numbers: (1107321,)
    run_numbers: (1107321,)
    pileups: (1107321,)


In [6]:
def plot_inputs(datasets, save_path, tags_to_plot):
    plt.rcParams['axes.linewidth'] = 2.4
    # HLT data -------------------------------------------------------------------------------------------------------------------------
    bin_dict = {
        'jets': {
            'pt': np.linspace(0, 1500, 35),
            'eta': np.linspace(-5, 5, 35),
            'phi': np.linspace(-np.pi, np.pi, 35)
        },

        'electrons': {
            'pt': np.linspace(0, 500, 35),
            'eta': np.linspace(-5, 5, 35),
            'phi': np.linspace(-np.pi, np.pi, 35)
        },

        'muons': {
            'pt': np.linspace(0, 500, 35),
            'eta': np.linspace(-5, 5, 35),
            'phi': np.linspace(-np.pi, np.pi, 35)
        },

        'photons': {
            'pt': np.linspace(0, 500, 35),
            'eta': np.linspace(-5, 5, 35),
            'phi': np.linspace(-np.pi, np.pi, 35)
        },

        'EMs': {
            'pt': np.linspace(0, 500, 35),
            'eta': np.linspace(-5, 5, 35),
            'phi': np.linspace(-np.pi, np.pi, 35)
        },

        'taus': {
            'pt': np.linspace(0, 500, 35),
            'eta': np.linspace(-5, 5, 35),
            'phi': np.linspace(-np.pi, np.pi, 35)
        },

        'MET': {
            'pt': np.linspace(0, 100, 35),
            'eta': np.linspace(-5, 5, 35),
            'phi': np.linspace(-np.pi, np.pi, 35)
        }
    }
    
    for obj_name, obj_data_range in zip(['jets', 'electrons', 'muons', 'photons'], [(0, 6), (6, 9), (9, 12), (12, 15)]):
        for i, var_name in enumerate(['pt', 'eta', 'phi']):
            bins = bin_dict[obj_name][var_name]
            plt.figure(figsize=(14, 8))
            for tag in tags_to_plot:
                data = datasets[tag][f'HLT_data']
                weights = datasets[tag]['weights']
                obj_data = data[:, obj_data_range[0]:obj_data_range[1], :]
                # Mask out zero values
                non_zero_mask = obj_data[:, :, i].flatten() != 0
                non_zero_data = obj_data[:, :, i].flatten()[non_zero_mask]
                non_zero_weights = np.repeat(weights, obj_data.shape[1])[non_zero_mask]
                plt.hist(non_zero_data, bins=bins, label=tag, histtype='step', density=True, fill=False, linewidth=3, weights=non_zero_weights, alpha=0.5)
            plt.title(f'HLT {obj_name} {var_name} Distribution (All Objects)')
            if var_name == 'pt': 
                plt.yscale('log')
            plt.xlabel(var_name)
            plt.ylabel('Normalized Frequency')
            plt.legend()
            plt.savefig(os.path.join(save_path, f'HLT_{obj_name}_{var_name}_all.png'))
            plt.clf()
            
            # For leading object
            plt.figure(figsize=(14, 8))
            for tag in tags_to_plot:
                data = datasets[tag][f'HLT_data']
                weights = datasets[tag]['weights']
                obj_data = data[:, obj_data_range[0]:obj_data_range[1], :]
                # Mask out zero values
                non_zero_mask = obj_data[:, 0, i] != 0
                non_zero_data = obj_data[:, 0, i][non_zero_mask]
                non_zero_weights = weights[non_zero_mask]
                plt.hist(non_zero_data, bins=bins, label=tag, histtype='step', density=True, fill=False, linewidth=3, weights=non_zero_weights, alpha=0.5)
            plt.title(f'HLT {obj_name} {var_name} Distribution (Leading Object)')
            if var_name == 'pt': 
                plt.yscale('log')
            plt.xlabel(var_name)
            plt.ylabel('Normalized Frequency')
            plt.legend()
            plt.savefig(os.path.join(save_path, f'HLT_{obj_name}_{var_name}_leading.png'))
            plt.clf()

    # MET plots
    for i, var_name in enumerate(['pt', 'eta']):
        bins = bin_dict['MET'][var_name]
        plt.figure(figsize=(14, 8))
        for tag in tags_to_plot:
            data = datasets[tag][f'HLT_data']
            weights = datasets[tag]['weights']
            MET = data[:, -1, :]
            # Mask out zero values
            non_zero_mask = MET[:, i] != 0
            non_zero_MET = MET[:, i][non_zero_mask]
            non_zero_weights = weights[non_zero_mask]
            plt.hist(non_zero_MET, bins=bins, label=tag, histtype='step', density=True, fill=False, linewidth=3, weights=non_zero_weights, alpha=0.5)
        plt.title(f'HLT MET {var_name} Distribution')
        if var_name == 'pt': 
                plt.yscale('log')
        plt.xlabel(var_name)
        plt.ylabel('Normalized Frequency')
        plt.legend()
        plt.savefig(os.path.join(save_path, f'HLT_MET_{var_name}.png'))
        plt.clf()

    # L1 data # ------------------------------------------------------------------------------------------------------------------------
    for obj_name, obj_data_range in zip(['jets', 'EMs', 'muons', 'taus'], [(0, 6), (6, 9), (9, 12), (12, 15)]):
        for i, var_name in enumerate(['pt', 'eta', 'phi']):
            bins = bin_dict[obj_name][var_name]
            plt.figure(figsize=(14, 8))
            for tag in tags_to_plot:
                data = datasets[tag]['L1_data']
                data[:, 9:12, 0] *= 1000 # Get the L1 muons into the right units
                weights = datasets[tag]['weights']
                obj_data = data[:, obj_data_range[0]:obj_data_range[1], :]
                # Mask out zero values
                non_zero_mask = obj_data[:, :, i].flatten() != 0
                non_zero_data = obj_data[:, :, i].flatten()[non_zero_mask]
                non_zero_weights = np.repeat(weights, obj_data.shape[1])[non_zero_mask]
                plt.hist(non_zero_data, bins=bins, label=tag, histtype='step', density=True, fill=False, linewidth=3, weights=non_zero_weights, alpha=0.5)
            plt.title(f'L1 {obj_name} {var_name} Distribution (All Objects)')
            if var_name == 'pt': 
                plt.yscale('log')
            plt.xlabel(var_name)
            plt.ylabel('Normalized Frequency')
            plt.legend()
            plt.savefig(os.path.join(save_path, f'L1_{obj_name}_{var_name}_all.png'))
            plt.clf()
            
            # For leading object
            plt.figure(figsize=(14, 8))
            for tag in tags_to_plot:
                data = datasets[tag]['L1_data']
                weights = datasets[tag]['weights']
                obj_data = data[:, obj_data_range[0]:obj_data_range[1], :]
                # Mask out zero values
                non_zero_mask = obj_data[:, 0, i] != 0
                non_zero_data = obj_data[:, 0, i][non_zero_mask]
                non_zero_weights = weights[non_zero_mask]
                plt.hist(non_zero_data, bins=bins, label=tag, histtype='step', density=True, fill=False, linewidth=3, weights=non_zero_weights, alpha=0.5)
            plt.title(f'L1 {obj_name} {var_name} Distribution (Leading Object)')
            if var_name == 'pt': 
                plt.yscale('log')
            plt.xlabel(var_name)
            plt.ylabel('Normalized Frequency')
            plt.legend()
            plt.savefig(os.path.join(save_path, f'L1_{obj_name}_{var_name}_leading.png'))
            plt.clf()

    # MET plots
    for i, var_name in enumerate(['pt', 'eta']):
        bins = bin_dict['MET'][var_name]
        plt.figure(figsize=(14, 8))
        for tag in tags_to_plot:
            data = datasets[tag]['L1_data']
            weights = datasets[tag]['weights']
            MET = data[:, -1, :]
            # Mask out zero values
            non_zero_mask = MET[:, i] != 0
            non_zero_MET = MET[:, i][non_zero_mask]
            non_zero_weights = weights[non_zero_mask]
            plt.hist(non_zero_MET, bins=bins, label=tag, histtype='step', density=True, fill=False, linewidth=3, weights=non_zero_weights, alpha=0.5)
        plt.title(f'L1 MET {var_name} Distribution')
        if var_name == 'pt': 
                plt.yscale('log')
        plt.xlabel(var_name)
        plt.ylabel('Normalized Frequency')
        plt.legend()
        plt.savefig(os.path.join(save_path, f'L1_MET_{var_name}.png'))
        plt.clf()

In [7]:
tags_to_plot = [tag for tag in datasets.keys() if tag.startswith('EB')]
plot_inputs(datasets, save_path = '/eos/home-m/mmcohen/ad_trigger_development/plots/feature_plots_11-11-2024', tags_to_plot=tags_to_plot)

/tmp/ipykernel_485/4191292515.py:51: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(14, 8))
/cvmfs/sft.cern.ch/lcg/views/LCG_105a_cuda/x86_64-el9-gcc11-opt/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges
/tmp/ipykernel_485/4191292515.py:129: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale('log')
/tmp/ipykernel_485/4191292515.py:149: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale('log')


<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

...

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>

<Figure size 1008x576 with 0 Axes>